# Conversion Analysis with Google Analytics Data

This sample notebook demonstrates working with Google Analytics page views and session data exported to BigQuery.

Google Analytics offers BigQuery export as part of its premium offering. If you're a premium user, you already have the ability to export any of your analytics views to a BigQuery dataset you own. If you're not, you can use the Analytics API to retrieve and import the data used to generate the default Analytics dashboards.

The sample data used in this notebook is shares the same schema as the Google Analytics BigQuery export, but is from a sample account, and hence available publicly. However it is also tiny in terms of size. This notebook demonstrates one possible custom analytics scenario, rather than the actual data.

Related Links:

* [BigQuery](https://cloud.google.com/bigquery/what-is-bigquery)
* [Google Analytics](https://analytics.google.com/)
* [Google Charting API](https://developers.google.com/chart/?hl=en) for data visualization

In [1]:
import gcp.bigquery as bq

# Understanding the Hits Data

Its helpful to inspect the schema and a sample of the data we're working with.

In [2]:
%%bigquery schema --table "google.com:analytics-bigquery:LondonCycleHelmet.ga_sessions_20130910"

The Google Analytics dataset has quite a large schema. It should be interesting to see some of the data in the important columns.

In [3]:
%%sql
SELECT fullVisitorId, visitId, hits.hitNumber AS hitNumber, hits.page.pagePath AS path
FROM [google.com:analytics-bigquery:LondonCycleHelmet.ga_sessions_20130910]
ORDER BY visitStartTime, hits.hitNumber

fullVisitorId,visitId,hitNumber,path
2879713562608983525,1378803173,1,/
2879713562608983525,1378803173,2,/vests/
2879713562608983525,1378803173,3,/vests/orange.html
2879713562608983525,1378803173,4,/vests/orange.html
2879713562608983525,1378803173,5,/
2879713562608983525,1378803173,6,/vests/
2879713562608983525,1378803173,7,/vests/yellow.html
2879713562608983525,1378803173,8,/vests/yellow.html
2879713562608983525,1378803250,1,/helmets/
6955732612793949850,1378803356,1,/helmets/heavy.html


The data is organized as a set of visits (or sessions), with each visit containing a set of hits (or page views) in succession. Each hit has a URL path associated with it. Here is another query that shows paths, and the number of hits across sessions.

In [4]:
%%sql
SELECT hits.page.pagePath as path, COUNT(visitId) as hits
FROM [google.com:analytics-bigquery:LondonCycleHelmet.ga_sessions_20130910]
GROUP BY path
ORDER BY hits DESC

path,hits
/confirm.html,51
/,45
/login.html,43
/helmets/foldable.html,38
/vests/yellow.html,24
/vests/,23
/helmets/,23
/basket.html,21
/vests/orange.html,20
/shipping.html,20


# Producing Conversion Data

For the purposes of this sample, the question to be answered is which path leads to higher conversion ratio - users landing on the landing page (path = '/') or users landing on a product page (eg. '/vests/yellow.html'). Conversion itself is defined as the user loading the '/confirm' page within a single session.

In [5]:
%%sql --module conversions

DEFINE QUERY AnnotatedVisits
SELECT
  visitId, hits.page.pagePath AS path, hits.hitNumber AS hitNumber,
  SUM(IF(hits.page.pagePath == '/confirm.html', 1, 0)) WITHIN RECORD AS transacted
FROM [google.com:analytics-bigquery:LondonCycleHelmet.ga_sessions_20130910]
ORDER BY visitStartTime, hitNumber

SELECT
  IF (path = '/', 'home', 'product') AS start,
  IF (transacted <> 0, 'completed', 'abandoned') AS outcome,
  COUNT(*) AS count
FROM $AnnotatedVisits
WHERE hitNumber = 1
GROUP BY start, outcome

In [6]:
%%bigquery execute --query conversions

start,outcome,count
home,abandoned,18
product,abandoned,29
product,completed,11
home,completed,5


## Visualizing the Conversion Path

The matrix tells us a bit about completed visits vs. abandoned visits depending on the starting point. However this is much more easily seen in a [sankey diagram](https://developers.google.com/chart/interactive/docs/gallery/sankey), which is provided by the Google Charting API.

In [7]:
%%chart sankey --data conversions
{
  "sankey": {
    "node": {
      "colors": [ "black", "red", "black", "green" ]
    }
  }
}

## Insights

The results and the diagram tell that the product pages lead to a slightly better conversion.

# Going Beyond the Sample

This sample demonstrated how you can apply the power of BigQuery to your Google Analytics, and more generally to any analytics telemetry data. Essentially you can build custom metrics, visualization and track the behavior and engagement of your audience even more effectively by joining analytics data with your own data.